In [1]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec, clean_str
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine


In [2]:
line_numbers = [118492, 36837, 49862, 156657, 73210, 82614, 124589, 96774, 87351, 131900, 126641, 150820, 92317, 31611, 83771, 162407, 82805, 101984, 40352, 129194, 134468, 33804, 141364, 65853, 63495, 119419, 155245, 149698, 164738, 127063, 131675, 7843, 37892, 83384, 12951, 58347, 6516, 161370, 90877, 34785, 38771, 162116, 153821, 116162, 108963, 71227, 102613, 50546, 157394, 131286, 40669, 48566, 113336, 16802, 140822, 149605, 103657, 81552, 32262, 43580, 129770, 125744, 35873, 33230, 60100, 167813, 3610, 97197, 81399, 64430, 116753, 31588, 149633, 39569, 129842, 5137, 124808, 23553, 13798, 66166, 53996, 116903, 95024, 94205, 31034, 127892, 42159, 145004, 23262, 131782, 107771, 66779, 84275, 78698, 11425, 33664, 4431, 96970, 129457, 73703, 126157, 145712, 59470, 32418, 145621, 168733, 67952, 105066, 81933, 67545, 139230, 22173, 105019, 8904, 43334, 43393, 129793, 142893, 114329, 106325, 115012, 23801, 101856, 6681, 7813, 129187, 121067, 160941, 129181, 134745, 107019, 70442, 127677, 103844, 139582, 6914, 154795, 16207, 150943, 123641, 70181, 126540, 66422, 30397, 6641, 95147, 109982, 126016, 33645, 154658, 7151, 59941, 10743, 16546, 78659, 33375, 72363, 165081, 20528, 33952, 75728, 54390, 100037, 111232, 67986, 92728, 66577, 133289, 71120, 76875, 80207, 15708, 33084, 87176, 79771, 6029, 110388, 157367, 153523, 154298, 143869, 69012, 11820, 76039, 57733, 106812, 12962, 115623, 18068, 158420, 165128, 119616, 18091, 18536, 168169, 118814, 3203, 145340, 31959, 27436, 132706, 59798, 107358, 164881, 75367, 137338, 87531, 13996, 140826, 85495, 70043, 74634, 86991, 108818, 160565, 166203, 33497, 91348, 56205, 53448, 166444, 150935, 60643, 104370, 115724, 9988, 145954, 72126, 46355, 17538, 19807, 21201, 42982, 116896, 112341, 50181, 64095, 109819, 56281, 67359, 157531, 76720, 13268, 141, 129166, 91165, 63148, 102973, 111475, 83592, 80625, 10669, 76212, 80059, 111970, 5039, 95963, 105102, 46423, 59403, 13306, 848, 45002, 5456, 45934, 15194, 45004, 77472, 17699, 127341, 35304, 147683, 116570, 110168, 40431, 100678, 72179, 40707, 14683, 164946, 141692, 57977, 110181, 67603, 148932, 123891, 143446, 7525, 115293, 43243, 67899, 165022, 46057, 109322, 33192, 158274, 39428, 151944, 114127, 145349, 160960, 36662, 139574, 105396, 114649, 63575, 79223, 150822, 41314, 97379, 56813, 80439, 20729, 128943, 163735, 59982, 159836, 83226, 66116, 164079, 5294, 101270, 114123, 63496, 15785, 41846, 109986, 42229, 113422, 133293, 70946, 91216, 121735, 7107, 50172, 69671, 53457, 95725, 82690, 16688, 135906, 26829, 42736, 114268, 168996, 122113, 2769, 95186, 117519, 45252, 53161, 72037, 121882, 59956, 14437, 95866, 102384, 117680, 162665, 47458, 65891, 163575, 156686, 473, 47386, 165587, 9183, 29142, 25675, 45261, 47396, 68126, 17511, 28799, 166963, 168619, 19560, 141628, 59556, 71231, 45525, 70151, 87747, 146131, 79472, 132890, 151889, 18375, 62053, 113177, 39453, 81667, 2970, 142045, 90954, 124438, 2437, 156835, 120825, 11338, 37873, 21883, 102887, 92223, 161555, 121029, 159669, 119955, 138750, 89859, 26195, 36418, 63991, 131662, 69883, 114715, 152681, 85988, 59682, 124622, 74726, 92115, 51986, 107108, 134934, 147474, 137394, 56466, 11538, 8088, 16789, 91199, 58126, 8740, 84949, 58187, 124654, 145413, 48916, 100106, 100072, 100265, 94880, 16337, 136866, 104899, 71012, 130574, 141032, 34379, 44293, 10015, 113240, 63471, 100266, 162062, 41763, 83089, 71517, 149540, 14602, 39622, 112715, 24820, 59323, 155610, 25635, 148453, 53925, 156345, 92596, 153579, 111593, 22441, 67622, 70777, 114525, 95694, 114983, 17072, 4185, 104471, 10381, 79724, 16216, 45041, 89532, 14541, 18654, 42869, 23928, 19709, 53669, 124261, 81216, 93385, 129312, 166355, 168674, 130686, 98517, 126852, 120808, 7646, 51263, 146239, 79091, 89244, 53921, 108663, 27762, 145785, 61945, 121555, 163330, 43045, 97857, 140652, 420, 52754, 45331, 89459, 109230, 27236, 60210, 99698, 105540, 52815, 85854, 22032, 123768, 117845, 34300, 140206, 45989, 142938, 72768, 143422, 107986, 166569, 98225, 156678, 76994, 161458, 153790, 118921, 39519, 144919, 140561, 157589, 46223, 155998, 149972, 146266, 81728, 137408, 143000, 129470, 139157, 105718, 122217, 143979, 88791, 130464, 9606, 83980, 91962, 162120, 157503, 157372, 164586, 34546, 159691, 61684, 97641, 21264, 12824, 122018, 53634, 12139, 121848, 14001, 99965, 72116, 34460, 57881, 163963, 13915, 149097, 112290, 163514, 52214, 94522, 156419, 5639, 129106, 165561, 56448, 77823, 85295, 127765, 16243, 98348, 35646, 158707, 62196, 59515, 168843, 24027, 114868, 101651, 69268, 150706, 145371, 11842, 164966, 66068, 95809, 133874, 124629, 115300, 101612, 102429, 116794, 46352, 120741, 80935, 145518, 93274, 49022, 159492, 166012, 141460, 128181, 151186, 119957, 150472, 107150, 94843, 128419, 73507, 149527, 30709, 70861, 54460, 110183, 60357, 32965, 76740, 104972, 79699, 148601, 137755, 117526, 46363, 62595, 150619, 39197, 68625, 113056, 119289, 59935, 55589, 1261, 34214, 4540, 53024, 8761, 118295, 144139, 143676, 49426, 80854, 138972, 113242, 64404, 11649, 101347, 128715, 50756, 12957, 77929, 23616, 83058, 157959, 51476, 58823, 107347, 93130, 90740, 39715, 48950, 13613, 149565, 70891, 124037, 159875, 123609, 131805, 57368, 121040, 1188, 153389, 39887, 93596, 103393, 16855, 52150, 60671, 165860, 4406, 163990, 117740, 89132, 111955, 146925, 7614, 111317, 666, 74671, 32847, 77200, 111958, 164900, 167421, 83788, 56730, 42796, 37465, 109513, 86659, 3885, 75305, 101494, 94639, 108125, 166659, 33789, 105371, 80564, 167043, 118415, 19011, 707, 94921, 63600, 104555, 81156, 26406, 103728, 81902, 55459, 97856, 136119, 95845, 59817, 100664, 158782, 136725, 25456, 115534, 92863, 9546, 139854, 164709, 112592, 60035, 162819, 165544, 39283, 133591, 124138, 10374, 130864, 31463, 132372, 138277, 98981, 83566, 92363, 97236, 27791, 157891, 56886, 133936, 50831, 148102, 139455, 133863, 95312, 62029, 152940, 29027, 5234, 123235, 162980, 109740, 144080, 7328, 128880, 64668, 74850, 25172, 98350, 125732, 7135, 83372, 13674, 40836, 19892, 56708, 4508, 159380, 32767, 32103, 136626, 160964, 167499, 73956, 6087, 10796, 29139, 43190, 103008, 53746, 106603, 127590, 121323, 124056, 149437, 149720, 30884, 129117, 128900, 71820, 74822, 96559, 141017, 73108, 40814, 146829, 162580, 370, 121159, 25397, 14521, 122682, 163011, 56996, 33864, 156095, 131668, 120823, 131589, 116931, 5599, 45054, 55449, 157742, 15137, 988, 84239, 60453, 136544, 94898, 166460, 148701, 87870, 161544, 64265, 161016, 5887, 61064, 2641, 4204, 23749, 154606, 142302, 57504, 161994, 143994, 115492, 65166, 87466, 162085, 83103, 35921, 111840, 21056, 75402, 124746, 38684, 103013, 153128, 67176, 147863, 154772, 163247, 30774, 107482, 64474, 27056, 2020, 46599, 39215, 74341, 67151, 94304, 50171, 48363, 9433, 77462, 41705, 66629, 147357, 124255, 139754, 61436, 153719, 39789, 28530, 41417, 10363, 75656, 105504, 80552, 147215, 121592, 126329, 132178, 12067, 41465, 113084, 41033, 82821, 58068, 108075, 112489, 44199, 161488, 5983, 114494, 111678, 74790, 84536, 102775, 66259, 93411, 103226, 78271, 94138, 12682, 143783, 98930, 83439, 131002, 9043, 62763, 98156, 166682, 97822, 142832, 120887, 103847, 67311, 953, 116726, 90431, 78066, 122356, 140087, 139336, 32021, 42437, 110727, 101507, 40852, 113266, 86019, 131050, 14909, 127370, 34986, 135604, 126328, 126202, 21121, 84488, 9164, 22624, 74643, 2554, 128869, 32236, 144364, 161007, 85911, 117245, 14173, 101569, 10849, 93187, 96854, 698, 92576, 75718, 159040, 117192, 60074, 141986, 33387, 67347, 118473, 113997, 133453, 91160, 114806, 120816, 35180, 161651, 130748, 18335, 48133, 63068, 25019, 141033, 73222, 145843, 38236, 74978, 46153, 35796, 69595, 8739, 65794, 132083, 18785, 4882, 104470, 30112, 130953, 87123, 78371, 25146, 38124, 132090, 133375, 140639, 65544, 123407, 138312, 70698, 92954, 49274, 44294, 11694, 107534, 137898, 80874, 113566, 28819, 40504, 158382, 61387, 4006, 9307, 58909, 134293, 140444, 97063, 110687, 7600, 53281, 159047, 38577, 25603, 80712, 102047, 35695, 145156, 81586, 105304, 88396, 129619, 158823, 116962, 141586, 31795, 72614, 161361, 166304, 120537, 38276, 38825, 1698, 41212, 143712, 91287, 1461, 84357, 44285, 54585, 105431, 37602, 14331, 60257, 39434, 133411, 13099, 30416, 141612, 105889, 96997, 149531, 68446, 64889, 47448, 49990, 23998, 70432, 150984, 15732, 8379, 13304, 74332, 5787, 140917, 32715, 117894, 114143, 131742, 128301, 35802, 34728, 30690, 116863, 68437, 140853, 65833, 19095, 39300, 103627, 78065, 135508, 138401, 89513, 150167, 120926, 54442, 30945, 106729, 56116, 38747, 126378, 121274, 1665, 163525, 55383, 24833, 147492, 92396, 45217, 79928, 88765, 26479, 31530, 66996, 117259, 44939, 99069, 42854, 112869, 86335, 146591, 150806, 6143, 77604, 43141, 57906, 98813, 122061, 32602, 32635, 118905, 145533, 76264, 143270, 4911, 35591, 127810, 114446, 105603, 13763, 96527, 145541, 26119, 132081, 140569, 44852, 63953, 44501, 13540, 85120, 4966, 50189, 147488, 1080, 20713, 104322, 51770, 45680, 44081, 22094, 25941, 40597, 149810, 64316, 93918, 93135, 102042, 130990, 50949, 92502, 79755, 78487, 141656, 105736, 152950, 66527, 78400, 48620, 111197, 60859, 143140, 116649, 46284, 112837, 11254, 122717, 88539, 115610, 167549, 131855, 23209, 142919, 157753, 52762, 28594, 2918, 127914, 9047, 30737, 147766, 64751, 48873, 51482, 13910, 1386, 69540, 117182, 51453, 108975, 96397, 80805, 23670, 143598, 65605, 40004, 44283, 123074, 40999, 150127, 113675, 165326, 13928, 52575, 166098, 12927, 156820, 31731, 90843, 103311, 26183, 147026, 98303, 8108, 75362, 40173, 142111, 49141, 9339, 84796, 121275, 41228, 9323, 108589, 74416, 92968, 18425, 48660, 147216, 127660, 91926, 36207, 153631, 7534, 79385, 147234, 162365, 72342, 120676, 140115, 142734, 104968, 89462, 26557, 150034, 151667, 117178, 72689, 29748, 42083, 154994, 162244, 161039, 106207, 2077, 50909, 18578, 61396, 95967, 55617, 155879, 72391, 65782, 70407, 84494, 162977, 141349, 71454, 126216, 63211, 74948, 133876, 141916, 141353, 95380, 15806, 43156, 83348, 46041, 123373, 90782, 115219, 123454, 60324, 120770, 35028, 56837, 22408, 161622, 119494, 4445, 150125, 39792, 141716, 92235, 48320, 119585, 60868, 162369, 69482, 113194, 132688, 27877, 63967, 97164, 8504, 58897, 37310, 73261, 151105, 102548, 123238, 121119, 329, 137158, 78302, 25136, 15432, 125823, 76793, 10258, 165910, 154860, 100887, 64744, 55333, 64327, 95828, 167176, 32054, 133947, 15037, 19642, 63162, 96764, 81220, 136035, 21754, 49537, 76540, 19774, 52218, 12370, 138483, 112545, 27618, 69907, 151091, 114814, 160861, 20027, 31651, 81971, 158004, 149383, 61317, 44678, 49727, 9159, 77981, 47397, 65722, 127762, 33728, 99455, 117135, 128328, 113809, 147915, 97407, 139730, 114069, 85399, 29010, 97940, 60736, 85590, 117143, 43526, 75608, 116987, 83816, 161933, 156391, 102946, 167023, 167819, 119285, 57324, 43125, 72532, 142244, 15486, 97488, 140623, 19999, 77836, 111771, 116229, 32651, 5546, 109284, 114839, 65617, 106674, 133225, 105799, 1484, 124117, 119092, 157658, 17330, 105119, 159153, 56775, 111858, 79322, 108562, 92455, 80824, 41194, 155349, 154838, 67811, 134629, 138812, 15566, 60340, 117188, 49961, 39492, 71910, 67460, 24041, 74898, 529, 29270, 97816, 160692, 591, 62188, 58072, 46928, 36871, 47399, 141317, 143406, 127008, 119403, 21763, 8796, 6796, 134419, 133414, 43200, 25466, 77701, 138434, 10919, 56814, 124074, 45153, 124911, 67907, 82325, 81450, 86438, 161554, 142867, 47973, 64266, 51029, 31694, 128393, 13318, 146159, 110292, 18130, 123712, 44620, 16685, 167090, 76511, 59205, 21452, 136158, 70639, 135386, 167334, 36561, 74781, 57944, 43958, 142963, 18382, 44629, 31846, 16085, 85746, 36058, 1774, 96409, 143870, 146306, 73856, 32982, 68945, 88412, 20451, 82797, 14138, 78253, 86089, 16840, 77973, 95125, 14281, 74286, 148541, 81991, 83941, 129190, 126163, 137172, 152872, 63443, 22994, 113000, 93569, 88820, 71495, 119767, 136702, 86343, 4855, 80880, 55899, 2074, 28715, 123550, 99152, 30714, 57262, 104637, 11711, 143091, 66125, 141529, 149740, 146285, 148264, 18898, 149394, 137162, 54580, 60115, 42846, 75703, 29147, 9432, 79111, 20115, 16775, 41652, 60764, 511, 25065, 138221, 90030, 169255, 57570, 63209, 27129, 129983, 11405, 37021, 161135, 153632, 28056, 15572, 15890, 23997, 83674, 129953, 36358, 111281, 80882, 161200, 86483, 168916, 57370, 145873, 109265, 112058, 109517, 156471, 144764, 117026, 75894, 130867, 26211, 134125, 65661, 160799, 84134, 62845, 4632, 106887, 57869, 108236, 106245, 9763, 112778, 708, 159660, 129899, 9567, 119136, 167860, 39281, 31426, 65393, 4277, 151920, 85894, 25915, 110738, 138046, 31988, 13080, 164263, 607, 39345, 35703, 60032, 121840, 19386, 22934, 84878, 151814, 107641, 24656, 166994, 57323, 130674, 29470, 144732, 122615, 159839, 92578, 19198, 25162, 52451, 58737, 9570, 91970, 8205, 134157, 42868, 7512, 12318, 123771, 127640, 46090, 139743, 163459, 161277, 44037, 89773, 20633, 127730, 120760, 82328, 166756, 137971, 130455, 47986, 48221, 87222, 10502, 146193, 25044, 129750, 72680, 165867, 159652, 158696, 48380, 41684, 126186, 100796, 99012, 75467, 89619, 9356, 159165, 119905, 73167, 164387, 87202, 81454, 159717, 157233, 150888, 87472, 105734, 165329, 145552, 147905, 142600, 150188, 158969, 148600, 59850, 114324, 32617, 26213, 38980, 151906, 41316, 140187, 54285, 18310, 2605, 145187, 105310, 160551, 102022, 64416, 5828, 50474, 133367, 163550, 168507, 125156, 144259, 11266, 133667, 56499, 111544, 14256, 22707, 117748, 33079, 7786, 38154, 81413, 123199, 94224, 76619, 103537, 121590, 99895, 128225, 161310, 40434, 37512, 3509, 49875, 129618, 150641, 78410, 48718, 150558, 149962, 71189, 55112, 52844, 6629, 99447, 105587, 102322, 25138, 120616, 32845, 58969, 152524, 32484, 134892, 82365, 146940, 492, 119437, 14676, 149186, 86996, 24932, 50451, 153453, 91612, 127124, 64199, 33297, 72094, 120364, 100180, 117892, 22826, 6253, 131483, 113295, 85557, 135635, 128086, 154452, 37297, 100488, 60510, 129081, 116845, 82665, 37307, 87298, 67828, 129094, 139889, 74238, 153944, 6291, 131431, 60574, 139928, 94563, 17396, 160776, 161670, 168601, 128306, 165621, 129471, 108971, 107151, 95684, 111249, 144959, 117438, 98669, 53507, 114639, 135372, 58081, 160070, 1327, 572, 60452, 88335, 74117, 120897, 168765, 59897, 23630, 107946, 108057, 158698, 133773, 34520, 70632, 127386, 69654, 124018, 23308, 22508, 99878, 73208, 150903, 38696, 60925, 80069, 126783, 148206, 101166, 65283, 136176, 59425, 133265, 136338, 27594, 76691, 96731, 58994, 59584, 97636, 71736, 14265, 62699, 123395, 146964, 127110, 5986, 124936, 133636, 155066, 33516, 14893, 49668, 61327, 27719, 113174, 131393, 164075, 73000, 45291, 70259, 1763, 148815, 54462, 93776, 37809, 100984, 46542, 5550, 94134, 36394, 126485, 150467, 70947, 48116, 86598, 126303, 35121, 95065, 59870, 7295, 102957, 55056, 56702, 29987, 8823, 114935, 90883, 120072, 126988, 71652, 115797, 88551, 85121, 17687, 124856, 131652, 167382, 68051, 4626, 46657, 110587, 138448, 69854, 65105, 66690, 144022, 49423, 153028, 126473, 158805, 19486, 42529, 138995, 104480, 76867, 97913, 48999, 27930, 86001, 143558, 12628, 29997, 20348, 23771, 125356, 131673, 1127, 167727, 166100, 82714, 139171, 61174, 17275, 49780, 126699, 38760, 23901, 64418, 25289, 125595, 119196, 106558, 153803, 119923, 2654, 96179, 87629, 159783, 65852, 91100, 27186, 41302, 136966, 14231, 78480, 154643, 133762, 46118, 151418, 30061, 15615, 90713, 3376, 79355, 52169, 106942, 118539, 18833, 166828, 56867, 93289, 57079, 7214, 106292, 92822, 36410, 96522, 88191, 35403, 126750, 131935, 59738, 121145, 89965, 144875, 124352, 137663, 165296, 55399, 32072, 103400, 31945, 20317, 73086, 149413, 131487, 79071, 93516, 142735, 103382, 31543, 160563, 35780, 142897, 69798, 128925, 58492, 80008, 168577, 34297, 163164, 51395, 19700, 108868, 39138, 132370, 99523, 109317, 167350, 38778, 141320, 163213, 161781, 42508, 132413, 30657, 2387, 98101, 148372, 63489, 13828, 30993, 124737, 1155, 59661, 9861, 42946, 52529, 123667, 14398, 63697, 30059, 56870, 88894, 17208, 81906, 25097, 134930, 86524, 165037, 154965, 6476, 81724, 143845, 19382, 129528, 77575, 74255, 78984, 44942, 102817, 53939, 22516, 28291, 141978, 28042, 94308, 96364, 59711, 82576, 103495, 46162, 124594, 11698, 113583, 44632, 108826, 121228, 135914, 142388, 59726, 84790, 33716, 4071, 78320, 97140, 148034, 40043, 70630, 27800, 5002, 74421, 123486, 92202, 133559, 62596, 40648, 49112, 16790, 54034, 87090, 115013, 157870, 71525, 106145, 3491, 135437, 24395, 168074, 74269, 86265, 152183, 116978, 66045, 11741, 85514, 130366, 2722, 135958, 127416, 1711, 110408, 3551, 91469, 69839, 163300, 64384, 152030, 104788, 119515, 4347, 54831, 94519, 137941, 167646, 90143, 90181, 59911, 51885, 159129, 17943, 54535, 13580, 50530, 162493, 57154, 154404, 92672, 28970, 160464, 11081, 63594, 68495, 90425, 46211, 53962, 14015, 166482, 100076, 153371, 139532, 34533, 99178, 89711, 131020, 33553, 110943, 33828, 132935, 50178, 118875, 161073, 98012, 56109, 165794, 78791, 23099, 149036, 22080, 116257, 15270, 28953, 7108, 23863, 49458, 25931, 69005, 87045, 67977, 34698, 27155, 147067, 25130, 63889, 108511, 35044, 7203, 142799, 60646, 163538, 76225, 86987, 88725, 163197, 65067, 156940, 25369, 162536, 113571, 140593, 104975, 112645, 36999, 94778, 47605, 31668, 141211, 32208, 98762, 158235, 134555, 1604, 111893, 118288, 105796, 124305, 51926, 98448, 106819, 78016, 97728, 121656, 56840, 3199, 74130, 167808, 66392, 31469, 63001, 107907, 52927, 156791, 91265, 75646, 144345, 15305, 22537, 33584, 145839, 77400, 121391, 65134, 671, 102436, 131701, 97971, 82608, 83882, 168143, 70277, 36854, 3674, 46029, 68409, 151166, 28494, 158743, 133092, 35341, 82836, 165474, 107992, 21808, 75423, 148176, 8939, 157470, 50006, 126576, 74324, 92420, 128669, 89599, 89732, 43418, 30887, 1053, 28613, 82453, 56028, 9153, 12912, 40792, 160424, 103117, 39999, 40943, 63537, 7838, 76827, 3559, 147420, 146969, 98940, 70553, 91009, 87114, 4938, 138149, 124860, 36241, 121669, 73323, 94210, 64243, 15722, 26235, 38161, 52515, 106560, 21866, 63409, 55386, 38064, 124431, 108290, 137551, 88695, 108474, 61669, 125191, 104812, 104174, 140841, 115398, 33931, 49641, 19779, 8431, 7414, 139701, 23176, 100348, 31207, 98041, 93505, 19462, 9170, 30005, 49177, 10507, 8579, 89279, 48048, 74492, 95954, 76917, 101288, 135874, 8022, 137744, 14024, 120619, 15508, 88559, 53863, 34100, 22787, 54837, 139654, 39516, 42188, 80304, 55619, 149476, 43105, 95777, 87921, 48036, 113990, 77304, 7619, 155509, 167915, 45099, 50444, 89321, 8422, 13527, 19980, 104076, 46441, 7807, 101746, 101511, 157610, 116051, 40450, 147374, 51600, 65527, 81285, 104130, 78848, 85720, 100287, 6483, 129264, 9224, 85843, 53127, 51062, 33709, 123879, 6861, 104533, 102725, 77394, 43447, 82196, 40532, 50984, 47030, 19351, 8009, 60950, 141779, 108174, 86341, 34683, 33705, 92045, 163647, 137983, 165667, 141178, 55298, 65558, 86697, 53650, 150789, 152330, 104867, 165585, 138154, 20674, 38535, 9723, 47119, 95112, 21336, 113931, 111192, 46549, 20741, 68209, 27120, 85176, 119772, 11330, 102036, 141492, 146541, 119550, 3122, 107109, 91463, 50516, 37175, 50155, 30253, 87886, 22534, 40201, 140489, 67535, 80692, 138252, 79746, 123228, 132575, 91965, 74710, 117407, 112911, 148318, 57388, 16432, 79262, 161466, 109185, 62112, 148987, 68206, 155450, 97711, 94879, 30008, 31823, 53243, 13681, 121095, 87813, 133416, 62923, 124086, 70250, 163954, 103378, 62616, 6632, 457, 79168, 128157, 40961, 143633, 76499, 30383, 83452, 32723, 135031, 65159, 5681, 5984, 20523, 47461, 42913, 55844, 13509, 114479, 43456, 156655, 147120, 122176, 164841, 38526, 46829, 117073, 53955, 145666, 48651, 60787, 8590, 82309, 55137, 53311, 71895, 73065, 61045, 139851, 160397, 38335, 134190, 35184, 157102, 87203, 137129, 58958, 156203, 40208, 26613, 115928, 9118, 44090, 151867, 118060, 154490, 55272, 60932, 168260, 74765, 128844, 150232, 79532, 76375, 78641, 32065, 73201, 79275, 123732, 156285, 148139, 2891, 153508, 47559, 138578, 107135, 149039, 91521, 73380, 52993, 149711, 44314, 100352, 46108, 52729, 168020, 12329, 65910, 9367, 149251, 113765, 109572, 92749, 75671, 148457, 104805, 31933, 103440, 45529, 123683, 151586, 45910, 156954, 165929, 39879, 132705, 119829, 108979, 20509, 17066, 108027, 32051, 88795, 135512, 34932, 23740, 79079, 34675, 140978, 153612, 105379, 157964, 38123, 126346, 122099, 168988, 85910, 133989, 22698, 126203, 166087, 102438, 65653, 13692, 78521, 120797, 28129, 12124, 138288, 50447, 128700, 35220, 74612, 93997, 68266, 74834, 95738, 4866, 59226, 38871, 111326, 42129, 139150, 31353, 81388, 45836, 131608, 50601, 7553, 74314, 140720, 59901, 114070, 58134, 160499, 113677, 49851, 16155, 49264, 98534, 1344, 44448, 151011, 143475, 73933, 54002, 101605, 152051, 36585, 65785, 70527, 131921, 76685, 46324, 140391, 56112, 12430, 164765, 141346, 168682, 78592, 89694, 32654, 136450, 158789, 6692, 129608, 124197, 68617, 84109, 135528, 86135, 93124, 165236, 119779, 146021, 33685, 77848, 47149, 10222, 119428, 137164, 159811, 39733, 3014, 28796, 10282, 80320, 125109, 157009, 109005, 7046, 45266, 79704, 122775, 141327, 87742, 49895, 103621, 105341, 48135, 101742, 57008, 24343, 10464, 125299, 124091, 59908, 1567, 18179, 29678, 108802, 17107, 56970, 134387, 68708, 157924, 168560, 149608, 91419, 56877, 161744, 134404, 147035, 51684, 12506, 167739, 132359, 165504, 74687, 109557, 2101, 116287, 155312, 144119, 67420, 134481, 143253, 14477, 154388, 56861, 126690, 166599, 123587, 129244, 30694, 42019, 84042, 44337, 31331, 104651, 39755, 19041, 62977, 158058, 16738, 151453, 64831, 29750, 21186, 161394, 3952, 151337, 72306, 70697, 39343, 133143, 71791, 73940, 56981, 75947, 74672, 134859, 168920, 45513, 7594, 8828, 104035, 157780, 98662, 135896, 143851, 8548, 12001, 52121, 52296, 141425, 76736, 150699, 70982, 153841, 112424, 145911, 38707, 99009, 136694, 60470, 70566, 2308, 4078, 95997, 164006, 143442, 24046, 42542, 144630, 95091, 144948, 150838, 111941, 44156, 32911, 20610, 69604, 104529, 153750, 56094, 115782, 77744, 124962, 31014, 94580, 40810, 100752, 55378, 22511, 48797, 109890, 130725, 25280, 66833, 141542, 127357, 2277, 67438, 93162, 9098, 137237, 47061, 55884, 105626, 29099, 30594, 88174, 82744, 43727, 153300, 8569, 38889, 106897, 154256, 163789, 10171, 144608, 51131, 26505, 84923, 12891, 105801, 142368, 19439, 32844, 130049, 24676, 142698, 60066, 143077, 65934, 157802, 162471, 61862, 29878, 142374, 122855, 162275, 81988, 11503, 10704, 123259, 88296, 27292, 32872, 19788, 84695, 25580, 40900, 75470, 54402, 147670, 112379, 45048, 154325, 163522, 47563, 65415, 43241, 167698, 7779, 160737, 34458, 74405, 154376, 66024, 26670, 130174, 35326, 112576, 160062, 80161, 148583, 165595, 74877, 156905, 139700, 167716, 120982, 67916, 130480, 19730, 85526, 160816, 58734, 136462, 108705, 140497, 56947, 124662, 22513, 23627, 146923, 167240, 132721, 104740, 162723, 155213, 75206, 128592, 144716, 161775, 10345, 50297, 164596, 141468, 165545, 4335, 30201, 70715, 164686, 141127, 26477, 120755, 57334, 1761, 52940, 159395, 58434, 5498, 137365, 29921, 131331, 125436, 43535, 160468, 28640, 79924, 104729, 131170, 147205, 8779, 76125, 3748, 60523, 64434, 68112, 5239, 152263, 103547, 150103, 106449, 10898, 12518, 87954, 37937, 63866, 99264, 92303, 111839, 33129, 123121, 80029, 13821, 28374, 58412, 118071, 84658, 122764, 23646, 34250, 137613, 78209, 109252, 16776, 150878, 157653, 107466, 115105, 125174, 113031, 92345, 12735, 3063, 109831, 115504, 148736, 29591, 67487, 137738, 11646, 134016, 148356, 91517, 130762, 52525, 159610, 125594, 1530, 105623, 147409, 21719, 122133, 74344, 75542, 121812, 140508, 30712, 53031, 121410, 141579, 150082, 11402, 112728, 61294, 130889, 5053, 58022, 7235, 105298, 89314, 129758, 163223, 98248, 376, 159577, 22797, 93098, 9784, 1731, 113223, 163273, 86359, 158744, 153014, 56098, 50076, 12079, 34618, 54939, 93111, 117811, 50576, 53317, 120675, 151912, 45051, 161690, 105125, 93860, 160634, 57348, 51785, 27945, 111623, 35812, 145442, 10138, 12059, 14441, 87284, 114652, 17142, 130101, 123766, 160228, 54073, 51206, 93, 102719, 80710, 97879, 127661, 83007, 136676, 85064, 13983, 19511, 168467, 12610, 99911, 147350, 119663, 103806, 72835, 19683, 132663, 34851, 123781, 75438, 131054, 72684, 111180, 62521, 146948, 34455, 75396, 122957, 18600, 30002, 48152, 103615, 71370, 4718, 168620, 25086, 81112, 139679, 121208, 68466, 148208, 139240, 160274, 83628, 75308, 16605, 20613, 165328, 56547, 5060, 119788, 37084, 51102, 116724, 143063, 94777, 54291, 66477, 80553, 126611, 2696, 93244, 102788, 102767, 14606, 149357, 58260, 136, 71918, 71128, 75777, 215, 77926, 132995, 18818, 138996, 15269, 138299, 51137, 168734, 73658, 53181, 146023, 88108, 75932, 88369, 148378, 27554, 124436, 81282, 106964, 92554, 28362, 28005, 42657, 46120, 68970, 163741, 159686, 66890, 156705, 96, 49079, 118301, 68710, 121537, 132737, 107871, 20805, 24952, 162458, 25239, 9775, 114542, 46038, 35612, 65817, 142404, 67049, 163867, 126304, 124606, 118434, 22457, 49279, 78672, 55803, 78304, 12897, 153466, 105798, 10451, 122346, 95296, 62337, 160921, 87359, 108111, 164426, 85756, 16295, 161623, 71745, 41676, 115832, 28430, 32949, 70055, 92016, 77897, 129787, 39680, 70925, 118419, 159384, 2366, 132329, 45139, 156517, 98100, 99151, 100306, 117020, 119222, 129433, 164206, 94392, 157709, 66048, 69021, 51584, 109660, 124872, 41659, 100626, 61479, 115161, 4156, 122823, 130287, 70460, 56887, 42496, 162254, 21181, 11066, 98714, 150315, 121665, 73544, 4433, 144601, 136322, 30659, 17640, 29517, 155485, 166651, 64606, 52593, 9598, 122331, 147033, 75216, 91106, 62447, 161266, 132104, 8789, 50385, 153914, 29230, 58970, 12101, 155737, 30024, 57772, 18329, 32934, 10671, 90650, 101003, 160824, 106040, 154574, 69431, 155335, 132228, 46191, 78310, 26448, 166789, 137773, 64485, 141518, 50567, 132634, 134295, 2099, 28505, 122830, 153275, 13528, 18758, 24396, 89825, 148720, 109037, 70133, 120283, 6352, 95941, 49056, 89327, 111878, 24466, 51546, 90603, 123295, 102972, 31536, 63213, 89723, 92264, 30413, 42973, 93046, 92770, 41642, 145869, 113688, 44972, 108508, 77019, 914, 65696, 130421, 46778, 96531, 91935, 62936, 134324, 81893, 89904, 142124, 107110, 19782, 75315, 42120, 12687, 160604, 14395, 160953, 28165, 121269, 45415, 48172, 159138, 134116, 59639, 167834, 77355, 80697, 168422, 159032, 133030, 125556, 95536, 93231, 38948, 33984, 10342, 114945, 130211, 85807, 36804, 150101, 34548, 52200, 101091, 136168, 137534, 121260, 122367, 5445, 6052, 109996, 17350, 83247, 158051, 44987, 167368, 28830, 109540, 36722, 168413, 80088, 101604, 74191, 28912, 11505, 121588, 51740, 69165, 132719, 78822, 57599, 122062, 7693, 27674, 44773, 106291, 27497, 103120, 113584, 48259, 66515, 52541, 5600, 128857, 42172, 121685, 75594, 106299, 31628, 163484, 122887, 9885, 23620, 8719, 165831, 94011, 72091, 66145, 120346, 121942, 156828, 62623, 12134, 39896, 127813, 72645, 135543, 54068, 146951, 132448, 68613, 149160, 130333, 122161, 37566, 123601, 105236, 24161, 123184, 101532, 159418, 102317, 162030, 49518, 51271, 71250, 115796, 112653, 31140, 49788, 113078, 92663, 149090, 138563, 54248, 146747, 166560, 74179, 81733, 143099, 70929, 65821, 121458, 136816, 153251, 32969, 87923, 139209, 24338, 142992, 73405, 149973, 14721, 74533, 152093, 29731, 119505, 155322, 93712, 152500, 71509, 44842, 102260, 118794, 6757, 167955, 145974, 43088, 94272, 52508, 71714, 130303, 21069, 67390, 162200, 120640, 54420, 133931, 32398, 160455, 167923, 164573, 79080, 123584, 97614, 154826, 98380, 120866, 108615, 134377, 62138, 51055, 115660, 136670, 60590, 60345, 147166, 89475, 86156, 81061, 136171, 157688, 50806, 37732, 128664, 75451, 26897, 14865, 65863, 67082, 76080, 73109, 142066, 61798, 108587, 166660, 161975, 78959, 93754, 126495, 17695, 158178, 111557, 51519, 83304, 124724, 8948, 31470, 67057, 160163, 27694, 90624, 43931, 164462, 23639, 148889, 2431, 137342, 7410, 114140, 1580, 163937, 151574, 143028, 98420, 146407, 97798, 138047, 3268, 150106, 150719, 4069, 3428, 157671, 82645, 131060, 34790, 116244, 120800, 13740, 58277, 105592, 58423, 114688, 127395, 87896, 6592, 64558, 124933, 49927, 94306, 85797, 157407, 157193, 95648, 139273, 159355, 121518, 4284, 161804, 137238, 160524, 91752, 18720, 157439, 168745, 159011, 24498, 42583, 122408, 160066, 159020, 117358, 57159, 121896, 153181, 29323, 101949, 109017, 23345, 155043, 80041, 34173, 166030, 123432, 111879, 138255, 90972, 154911, 135844, 147432, 50299, 60151, 31661, 158563, 114700, 3412, 130865, 151847, 25142, 123517, 151641, 47578, 53362, 23483, 80735, 149667, 113581, 66624, 95196, 129834, 82971, 15670, 5843, 156774, 47320, 162381, 166596, 40227, 147462, 105610, 101372, 10790, 135475, 64344, 97060, 65110, 100244, 38377, 36316, 144538, 18955, 117250, 43388, 71416, 14624, 93638, 166032, 130378, 93379, 75846, 68758, 23965, 65362, 145714, 135178, 138886, 120098, 72205, 41387, 54212, 63233, 157608, 135068, 30110, 148463, 11816, 112518, 97345, 23301, 102603, 10717, 146661, 53302, 84371, 85479, 128534, 94372, 167508, 66570, 84015, 95392, 146511, 40039, 132535, 116614, 387, 27831, 82856, 147900, 32998, 35353, 95909, 20538, 107852, 158601, 12234, 48439, 89518, 115234, 108978, 126806, 352, 158992, 74083, 70821, 11703, 56789, 25056, 14581, 80516, 106318, 71340, 127224, 51832, 69441, 40064, 109929, 43849, 60342, 74800, 159580, 1953, 19006, 161101, 36683, 97974, 72600, 125471, 28934, 153815, 110017, 29906, 102846, 64248, 103473, 45746, 108457, 28882, 75889, 43579, 123453, 95405, 124021, 155340, 35390, 36282, 75026, 150908, 11579, 107155, 45067, 99466, 39035, 63980, 84947, 43638, 75683, 7257, 28004, 119922, 164604, 119232, 21402, 64824, 100915, 120324, 88929, 113941, 73202, 117640, 17347, 36716, 6486, 22629, 69303, 106031, 5203, 49238, 50311, 73793, 147984, 127664, 116457, 78464, 30257, 35468, 15385, 120327, 88122, 117293, 100587, 58642, 79883, 36035, 163238, 100603, 4135, 34184, 140675, 145982, 139009, 162558, 150049, 47987, 121775, 20052, 25545, 57116, 150419, 109817, 85584, 71804, 148005, 155895, 26975, 17050, 157539, 121527, 119141, 111463, 18295, 148895, 46042, 155460, 3908, 40192, 102236, 169154, 124474, 74839, 166751, 116187, 127577, 149417, 137151, 133696, 149937, 163236, 61921, 131645, 35909, 50692, 4957, 16000, 2320, 97968, 64903, 90741, 10048, 128863, 77638, 76243, 147743, 110321, 24486, 52060, 140462, 109010, 112776, 56486, 15831, 42961, 25625, 61743, 78673, 33294, 73177, 42295, 1573, 83129, 62970, 36063, 89301, 62308, 42547, 32788, 32033, 125083, 118906, 147834, 125040, 156353, 57464, 62966, 118446, 39060, 71553, 129530, 7773, 134149, 51913, 68364, 24547, 31212, 95988, 146370, 127429, 43265, 153851, 100651, 146737, 35922, 55454, 5540, 94923, 70311, 145393, 45483, 121657, 89601, 66049, 16139, 18930, 161169, 143664, 139957, 120001, 136370, 133998, 150517, 20306, 88373, 21674, 89430, 86121, 42038, 132579, 22939, 47218, 132236, 138038, 107793, 168115, 10275, 168597, 166218, 44599, 20081, 34734, 1579, 14647, 50423, 99959, 69920, 119543, 146998, 102656, 85482, 117741, 89761, 50427, 77353, 149323, 128396, 71317, 132105, 58914, 130179, 89225, 102068, 59569, 125723, 383, 71212, 80142, 113515, 12881, 87189, 10418, 8044, 7953, 109591, 158032, 106884, 141527, 163033, 163692, 42057, 51976, 156434, 90804, 116014, 4197, 126109, 45909, 10482, 47142, 124990, 149344, 78929, 129121, 133294, 48279, 109964, 103758, 100957, 23743, 85732, 32736, 162049, 91315, 34105, 160661, 168190, 104094, 122603, 144673, 84203, 26625, 43249, 122193, 92421, 48789, 115385, 168140, 41424, 23696, 22022, 124028, 40821, 97386, 103483, 15242, 18047, 137317, 102038, 79962, 119728, 145134, 123613, 112903, 113033, 43765, 56854, 144179, 88067, 145558, 1742, 106475, 121162, 108714, 95272, 13104, 57366, 96566, 97377, 53210, 75239, 71286, 150007, 38828, 124646, 71534, 141863, 157725, 54488, 148227, 51927, 62261, 84144, 93107, 148221, 53692, 124865, 134908, 135552, 144177, 159449, 149342, 98212, 52591, 68292, 13420, 97811, 157053, 166971, 85693, 115240, 45034, 61734, 7387, 655, 123172, 14357, 50942, 125646, 69699, 131332, 87608, 147135, 148603, 75399, 154914, 27413, 6952, 75349, 156468, 74609, 30256, 166716, 136570, 3177, 137423, 44654, 61008, 61350, 37853, 136002, 152665, 63278, 124199, 47862, 68655, 110988, 109623, 62338, 119307, 1363, 67799, 140667, 165959, 4172, 76060, 75374, 158120, 20649, 118398, 80236, 80522, 34574, 163062, 29630, 85657, 113342, 142361, 144332, 21366, 77630, 58011, 36862, 18230, 134244, 17043, 105203, 155307, 94883, 57315, 98551, 96663, 147349, 39303, 122251, 34050, 57474, 59373, 78198, 122966, 144832, 86807, 107255, 139885, 99556, 153782, 136852, 12592, 146790, 93973, 159972, 28511, 150334, 25745, 164725, 15131, 39022, 29563, 153817, 132097, 52431, 27157, 85937, 63191, 94493, 31040, 101863, 140243, 34712, 139568, 86224, 19859, 77755, 99701, 114962, 32136, 44784, 31135, 153398, 4553, 146467, 113028, 98658, 41638, 39095, 148354, 166380, 83922, 31465, 24785, 101542, 155260, 140723, 46050, 75680, 28697, 167112, 30605, 136580, 67594, 110663, 30533, 67555, 89597, 74868, 134509, 75132, 161313, 162271, 9720, 81721, 44631, 71347, 66426, 153652, 59071, 33386, 91185, 47564, 65109, 166737, 80969, 51729, 58951, 12032, 3041, 119379, 59288, 4497, 157349, 167130, 140624, 90631, 49943, 137502, 150933, 117907, 12153, 15292, 24474, 32260, 21562, 43522, 140006, 130077, 167759, 110768, 114047, 32876, 15731, 156334, 139013, 111228, 28899, 89352, 105463, 48074, 118771, 34662, 62101, 152480, 145514, 57248, 103829, 17794, 77418, 6307, 42686, 61818, 132110, 10577, 153500, 83211, 159269, 75359, 142179, 56606, 67674, 25762, 14112, 138551, 21677, 972, 52368, 57633, 42143, 76987, 136842, 90304, 166377, 89061, 167775, 102907, 98201, 282, 41506, 121398, 124807, 47231, 158636, 103702, 157281, 152700, 163901, 20982, 29015, 162277, 3898, 48795, 45925, 59774, 66360, 72298, 96434, 161756, 84526, 357, 64039, 153433, 130613, 13130, 32915, 162622, 41621, 46494, 28823, 33340, 134504, 2990, 7804, 54586, 73948, 44838, 125210, 149911, 45445, 27226, 58862, 141839, 119454, 146228, 59307, 119943, 104638, 36549, 36606, 76909, 145804, 127315, 101283, 113087, 6241, 120818, 17983, 108426, 28660, 102367, 80872, 137410, 79200, 126124, 104823, 53349, 125977, 102377, 112885, 42329, 125915, 69312, 157617, 169211, 54283, 19610, 76071, 51040, 69067, 146371, 167372, 76564, 102829, 33328, 74617, 139017, 73095, 46280, 103353, 110561, 90722, 151853, 105206, 135360, 96625, 149256, 107532, 122046, 138917, 23126, 41935, 166176, 7866, 20794, 107543, 158071, 111367, 122074, 64928, 84487, 27135, 14252, 44360, 5718, 86705, 70901, 148927, 121811, 15396, 116585, 18943, 167850, 111350, 25607, 22736, 131009, 140139, 75371, 88434, 44749, 77694, 136091, 29508, 30198, 15942, 80731, 130217, 91262, 133214, 243, 59050, 145733, 59271, 165478, 13820, 98592, 133381, 45790, 152669, 9316, 62215, 131294, 108287, 156938, 10017, 57276, 25678, 94335, 103125, 114239, 33642, 1439, 51611, 87469, 64973, 30944, 141252, 129537, 50685, 156493, 125417, 133582, 152233, 27663, 7266, 102640, 107012, 136684, 1450, 2905, 103826, 1257, 107975, 29895, 10688, 152029, 147995, 115221, 100753, 40159, 83406, 99348, 102962, 137479, 46708, 138665, 30420, 69055, 95774, 77176, 67286, 104098, 158654, 20684, 147164, 159631, 46144, 23281, 123626, 83038, 141862, 77267, 114224, 123663, 87811, 50605, 89754, 25189, 117456, 150736, 27283, 164704, 48450, 23274, 167582, 138262, 18358, 50258, 67041, 30558, 125861, 17757, 22915, 57130, 36339, 18847, 109281, 133759, 83130, 55660, 128717, 124699, 28427, 166368, 68803, 142674, 63312, 118194, 690, 8279, 106981, 70552, 6225, 151235, 33648, 53752, 71293, 10343, 41513, 89835, 148974, 93814, 63942, 99957, 27034, 140961, 131273, 62972, 125818, 7155, 117711, 37674, 49299, 38386, 123633, 954, 7975, 132916, 103085, 129718, 42639, 148945, 149712, 54398, 100881, 14925, 31778, 15771, 89219, 48099, 168654, 18612, 163820, 127764, 55253, 127094, 91712, 26826, 27446, 152870, 70620, 161564, 92967, 102108, 53271, 86528, 74633, 132366, 40280, 19117, 86418, 26770, 95476, 30773, 135125, 74174, 133841, 89373, 60410, 59038, 2768, 26178, 62956, 160597, 156752, 62983, 35537, 63532, 145601, 105328, 82778, 36482, 106483, 120164, 161042, 143611, 167085, 4153, 93156, 128757, 93885, 98080, 5072, 10160, 141476, 113946, 5762, 108942, 135640, 135403, 90583, 9284, 68179, 106565, 84898, 83819, 35183, 142123, 65291, 27166, 121819, 115087, 61491, 48248, 136519, 117200, 39548, 109182, 2538, 5625, 122712, 85675, 106849, 35850, 73339, 87741, 110305, 159456, 85297, 103280, 39688, 96624, 166719, 110909, 98469, 85107, 1872, 117301, 4972, 161440, 146339, 69031, 15585, 3875, 103470, 129948, 90479, 30145, 163720, 35528, 54583, 97283, 1123, 4434, 59423, 129277, 123326, 82642, 18574, 138723, 135685, 150066, 146468, 131702, 151187, 143737, 159485, 87721, 123731, 114246, 146225, 168967, 44877, 119562, 104488, 154253, 38921, 141162, 31123, 16916, 125625, 32538, 168690, 12160, 104507, 64535, 205, 156644, 53407, 62344, 2367, 124198, 77714, 89371, 65267, 78880]

In [3]:
node_ids=[]
for line_no in line_numbers:
    node_id = line_no - 1
    node_ids.append(node_id)

node_ids.sort()

In [4]:
print(node_ids[:5])

[92, 95, 135, 140, 204]


In [5]:

word_embeddings_dim = 300
word_vector_map = {}

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open('data/all_labels.txt', 'r')
lines = f.readlines()

# Process only the lines with the given indices
for i, line in enumerate(lines):
    if i in node_ids:
        doc_name_list.append(line.strip())
        temp = line.split("\t")
        if temp[1].find('test') != -1:
            doc_test_list.append(line.strip())
        elif temp[1].find('train') != -1:
            doc_train_list.append(line.strip())

f.close()
print(doc_name_list[::500])
print(doc_train_list[::500])
print(doc_test_list[::500])

['92\ttrain\t34', '16774\ttrain\t5', '34172\ttrain\t23', '51130\tval\t28', '68178\ttrain\t10', '85478\ttrain\t24', '103012\ttrain\t16', '120639\ttrain\t20', '136034\ttrain\t2', '152182\ttrain\t4']
['92\ttrain\t34', '25606\ttrain\t16', '50446\ttrain\t37', '75358\ttrain\t3', '100347\ttrain\t27', '123640\ttrain\t5', '147473\ttrain\t16']
['140\ttest\t23', '115233\ttest\t14']


In [6]:

doc_content_list = []
f = open('data/corpus/clean_5000.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
print(doc_content_list[::500])


['action fingerprint liveness detection competition 2019 international fingerprint liveness detection competition \\( \\) open well acknowledged meeting point private companies deal problem distinguishing images coming fingerprints made artificial materials images relative real fingerprints competitors propose integrated algorithms matching systems goal investigate extent integration impact whole performance algorithms submitted competition , eight integrated systems', 'resnet one neuron hidden layers universal demonstrate deep resnet stacked modules one neuron per hidden layer relu activation functions uniformly approximate integrable function dimensions , e ell 1 \\( mathbb r \\) identity mapping inherent resnets , network alternating layers dimension one stands sharp contrast fully connected networks , universal approximators width input dimension lu et al , 2017 , 2017 hence , result implies increase representational power narrow deep networks resnet architecture', 'deep hashing ca

In [10]:
doc_content_list = []
f = open('data/corpus/clean_5000.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
# print(doc_content_list)

train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
print(train_ids)
random.shuffle(train_ids)

# partial labeled data
#train_ids = train_ids[:int(0.2 * len(train_ids))]

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('data/1500.train.index', 'w')
f.write(train_ids_str)
f.close()

test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
print(test_ids)
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('data/ind.1500.test.index', 'w')
f.write(test_ids_str)
f.close()

ids = train_ids + test_ids
print(ids)
print(len(ids))

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 16, 18, 20, 22, 23, 25, 26, 27, 32, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 90, 91, 92, 94, 96, 98, 99, 100, 101, 102, 104, 105, 107, 108, 109, 110, 112, 113, 114, 115, 117, 119, 122, 123, 124, 126, 128, 130, 133, 134, 135, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 160, 162, 164, 165, 167, 169, 170, 171, 172, 176, 178, 179, 180, 181, 182, 184, 186, 189, 190, 192, 193, 194, 195, 197, 198, 199, 202, 204, 207, 208, 209, 212, 213, 214, 215, 216, 217, 218, 220, 223, 225, 226, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 245, 246, 247, 248, 252, 253, 254, 255, 257, 258, 259, 261, 263, 264, 265, 268, 270, 271, 275, 276, 278, 279, 280, 281, 282, 283, 285, 286, 287, 288, 289, 292, 293, 294, 296, 300, 302, 303, 304, 305, 306, 310, 311, 313, 314, 

In [11]:
shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('data/shuffle_5000.txt', 'w')
f.write(shuffle_doc_name_str[::500])
f.close()

f = open('data/corpus/shuffle_5000.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()
print(shuffle_doc_name_list[::500])

['74340\ttrain\t32', '143978\ttrain\t16', '43199\ttrain\t30', '54836\ttrain\t24', '136543\ttrain\t2', '154657\ttrain\t16', '95271\ttrain\t24', '159685\ttest\t24', '81155\ttest\t3']


In [12]:

# build vocab
word_freq = {}
word_set = set()
for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)
vocab

['filling',
 'echo',
 'crowdsourced',
 'jt',
 'increment',
 'breaches',
 'look',
 'distance',
 'covariates',
 'actual',
 'ecosystem',
 'communicating',
 'may',
 'returns',
 'agnostic',
 'nouns',
 'inverse',
 'equivalent',
 'quantum',
 'forbidden',
 'fact',
 'twenty',
 'parts',
 'suboptimal',
 'pruning',
 'kolmogorov',
 'srl',
 'abductive',
 'proof',
 'serve',
 'contention',
 'quadratically',
 'mixed',
 'imbalance',
 'solvers',
 'dsm',
 'outsource',
 'descriptors',
 'characteristic',
 'lots',
 'explainability',
 'commonalities',
 'multilevel',
 'tight',
 'volumes',
 'sourced',
 'arxiv',
 'colluding',
 'corruption',
 'crowdfunding',
 'wallet',
 'microgrid',
 'plane',
 'scope',
 'handcrafted',
 'reversible',
 'arbitrary',
 'visit',
 'attentions',
 'efficiency',
 'wip',
 '128',
 'inversion',
 'mfgs',
 'varies',
 'freely',
 'mitigated',
 '4x',
 'debt',
 'abstracts',
 'offered',
 'bptt',
 'merges',
 'univariate',
 'overfitting',
 'ieee',
 'oracles',
 'journalism',
 'interpolant',
 'mental',


In [13]:

word_doc_list = {}

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)

word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

vocab_str = '\n'.join(vocab)

f = open('data/corpus/vocab_5000.txt', 'w')
f.write(vocab_str)
f.close()

In [11]:
# label list
label_set = set()
for doc_meta in shuffle_doc_name_list:
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

label_list_str = '\n'.join(label_list)
f = open('data/corpus/labels_1500.txt', 'w')
f.write(label_list_str)
f.close()

In [12]:
train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size  # - int(0.5 * train_size)
# different training rates

real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('data/real_train_1500.name', 'w')
f.write(real_train_doc_names_str)
f.close()

In [13]:

row_x = []
col_x = []
data_x = []
for i in range(real_train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            # print(doc_vec)
            # print(np.array(word_vector))
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_x.append(doc_vec[j] / doc_len)  # doc_vec[j]/ doc_len

# x = sp.csr_matrix((real_train_size, word_embeddings_dim), dtype=np.float32)
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))

y = []
for i in range(real_train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)
print(y)

# tx: feature vectors of test docs, no initial features
test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in range(test_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_tx.append(doc_vec[j] / doc_len)  # doc_vec[j] / doc_len

# tx = sp.csr_matrix((test_size, word_embeddings_dim), dtype=np.float32)
tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))

ty = []
for i in range(test_size):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)
print(ty)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [14]:
word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

row_allx = []
col_allx = []
data_allx = []

for i in range(train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_allx.append(doc_vec[j] / doc_len)  # doc_vec[j]/doc_len
for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)

allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))

ally = []
for i in range(train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

for i in range(vocab_size):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)



(944, 300) (944, 38) (228, 300) (228, 38) (5780, 300) (5780, 38)


In [15]:

# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

'''
Doc word heterogeneous graph
'''

# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)


word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

In [16]:




# word vector cosine similarity as weights

'''
for i in range(vocab_size):
    for j in range(vocab_size):
        if vocab[i] in word_vector_map and vocab[j] in word_vector_map:
            vector_i = np.array(word_vector_map[vocab[i]])
            vector_j = np.array(word_vector_map[vocab[j]])
            similarity = 1.0 - cosine(vector_i, vector_j)
            if similarity > 0.9:
                print(vocab[i], vocab[j], similarity)
                row.append(train_size + i)
                col.append(train_size + j)
                weight.append(similarity)
'''
# doc word frequency
doc_word_freq = {}

for doc_id in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[doc_id]
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))


In [17]:

# dump objects
f = open("data/ind.x", 'wb')
pkl.dump(x, f)
f.close()

f = open("data/ind.y", 'wb')
pkl.dump(y, f)
f.close()

f = open("data/ind.tx", 'wb')
pkl.dump(tx, f)
f.close()

f = open("data/ind.ty", 'wb')
pkl.dump(ty, f)
f.close()

f = open("data/ind.allx", 'wb')
pkl.dump(allx, f)
f.close()

f = open("data/ind.ally", 'wb')
pkl.dump(ally, f)
f.close()

f = open("data/ind.adj", 'wb')
pkl.dump(adj, f)
f.close()